In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
from importlib import reload
from datetime import datetime

sys.path.insert(0, '../src')
import deepgeo.dataset.data_augment as dtaug
import deepgeo.dataset.utils as dsutils 
import deepgeo.common.geofunctions as gf
import deepgeo.networks.model_builder as mb
import deepgeo.networks.loss_functions as lossf

reload(dtaug)
reload(dsutils)
reload(mb)
reload(gf)

/home/raian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'deepgeo.common.geofunctions' from '../src/deepgeo/common/geofunctions.py'>

# Load input Dataset

In [2]:
# DATA_DIR = os.path.join(os.path.abspath(os.path.dirname('__file__')), '../', 'data_real', 'generated')
network = 'unet'
fusion = 'none'
DATA_DIR = '/home/raian/doutorado/Dados/generated'

DATASET = os.path.join(DATA_DIR, 'dataset_316x316_w_non_forest_0-1-2013-2017/')
dataset_fldr = 'w_non_forest_neg1_2_1'

class_names = ['no data', 'not deforestation', 'deforestation', 'non forest', 'clouds']
# model_dir = os.path.join(DATA_DIR, 'tf_logs', 'experiments', 'unet', dataset_fldr, 'fusion_%s' % fusion,
#                          'test_%s_%s' % (network, datetime.now().strftime('%d_%m_%Y-%H_%M_%S')))
# model_dir = os.path.join(DATA_DIR, 'tf_logs', 'experiments', 'unet', 'def_one_class_SR',
#                              'fusion_%s' % fusion, 'test_unet_2019_07_25-22_07_50')
train_tfrecord = os.path.join(DATASET, 'dataset_train.tfrecord')
test_tfrecord = os.path.join(DATASET, 'dataset_test.tfrecord')
val_dataset = os.path.join(DATASET, 'dataset_valid.npz')

In [3]:
model_dir = '/home/raian/doutorado/deepgeo/data_real/generated/tf_logs/test_debug'
!rm -rf {model_dir}/*
!ls -al {model_dir}

total 8
drwxr-xr-x 2 raian digits 4096 Aug 23 16:55 .
drwxr-xr-x 3 raian digits 4096 Dec  7  2018 ..


## Computing weights

In [4]:
weights_train = lossf.compute_weights_mean_proportion(train_tfrecord, class_names, ['no data'])
weights_eval = lossf.compute_weights_mean_proportion(test_tfrecord, class_names, ['no data'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
print(weights_train)
print(weights_eval)

[0, 0.43837945686249175, 0.8142994851844507, 62.7101208631067, 2.1058154863896887]
[0, 0.4326019152353616, 0.8102183510632927, 92.29865102648057, 2.2556235115549335]


In [9]:
params = {
    'network': network,
    'fusion': fusion,
    'epochs': 1,
    'batch_size': 20,
    'chip_size': 316,
    'bands': 5,
    'filter_reduction': 0.5,
    'learning_rate': 0.1,
    'learning_rate_decay': True,
    'decay_rate': 0.95 ,
    'l2_reg_rate': 0.0005,
    # 'var_scale_factor': 2.0,  # TODO: Put the initializer as parameter
    'chips_tensorboard': 2,
    # 'dropout_rate': 0.5,  # TODO: Put a bool parameter to apply or not Dropout
    'loss_func': 'avg_soft_dice',
    'data_aug_ops': ['rot90', 'rot180', 'rot270', 'flip_left_right',
                     'flip_up_down', 'flip_transpose'],
    'data_aug_per_chip': 4,
    #'class_weights': {'train': weights_train, 'eval': weights_eval},
    'num_classes': len(class_names),
    'class_names': class_names,
    'num_compositions': 1,
    'bands_plot': [[1, 2, 3]], #, [6, 7, 8]],
    'Notes': 'Testing.'
}

# Train the Network

In [10]:
reload(mb)

<module 'deepgeo.networks.model_builder' from '../src/deepgeo/networks/model_builder.py'>

In [11]:
model = mb.ModelBuilder(params)
model.train(train_tfrecord, test_tfrecord, model_dir)

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_eval_distribute': None, '_experimental_distribute': None, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  r

In [ ]:
# dataset = np.load(val_dataset)
# model = mb.ModelBuilder(model_dir)
# model.validate(dataset['chips'], dataset['labels'], model_dir, exclude_classes=[])